In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn import  pipeline, preprocessing
from torchvision.datasets import EMNIST

2024-01-28 18:15:49.280474: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-28 18:15:49.308768: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 18:15:49.308796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 18:15:49.309473: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 18:15:49.314562: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-01-28 18:15:53.323269: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 18:15:53.324254: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 18:15:53.324370: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:


# Load the datasets
train_data = EMNIST(root='emnist_data/', split='byclass', train=True, download=True)
test_data = EMNIST(root='emnist_data/', split='byclass', train=False, download=True)
#unpack the data into images and labels
train_images, train_labels = train_data.data, train_data.targets
test_images, test_labels = test_data.data, test_data.targets

#convert the data to numpy arrays
train_images = train_images.numpy()
train_labels = train_labels.numpy()
test_images = test_images.numpy()
test_labels = test_labels.numpy()


In [4]:
type(train_images)

numpy.ndarray

In [5]:
features_preprocessing_pileline = pipeline.make_pipeline(
    preprocessing.FunctionTransformer(lambda x: tf.convert_to_tensor(x, dtype=tf.float32))
)

train_images = features_preprocessing_pileline.fit_transform(train_images)

train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int64)


2024-01-28 18:15:54.001216: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 18:15:54.001440: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-28 18:15:54.001516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
train_images.shape

TensorShape([697932, 28, 28])

In [7]:

model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(1024, activation="tanh"),
    layers.Dense(1024//2, activation="tanh"),
    layers.Dense(1024//4, activation="tanh"),
    layers.Dense(62, activation="softmax")
])

In [8]:

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [9]:

model.fit(train_images, train_labels, epochs=10, batch_size=64*2)

Epoch 1/10


2024-01-28 18:15:56.797381: I external/local_xla/xla/service/service.cc:168] XLA service 0x565008dd5c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-28 18:15:56.797414: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-01-28 18:15:56.814473: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-28 18:15:56.851635: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1706454956.970910    7310 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5453/5453 [==============================] - 13s 2ms/step - loss: 0.9817 - accuracy: 0.7000
Epoch 2/10
5453/5453 [==============================] - 11s 2ms/step - loss: 0.8224 - accuracy: 0.7380
Epoch 3/10
5453/5453 [==============================] - 11s 2ms/step - loss: 0.8000 - accuracy: 0.7434
Epoch 4/10
5453/5453 [==============================] - 11s 2ms/step - loss: 0.7911 - accuracy: 0.7464
Epoch 5/10
5453/5453 [==============================] - 11s 2ms/step - loss: 0.7837 - accuracy: 0.7482
Epoch 6/10
5453/5453 [==============================] - 12s 2ms/step - loss: 0.7799 - accuracy: 0.7482
Epoch 7/10
5453/5453 [==============================] - 13s 2ms/step - loss: 0.7781 - accuracy: 0.7498
Epoch 8/10
5453/5453 [==============================] - 11s 2ms/step - loss: 0.7792 - accuracy: 0.7494
Epoch 9/10
5453/5453 [==============================] - 11s 2ms/step - loss: 0.7690 - accuracy: 0.7518
Epoch 10/10
5453/5453 [==============================] - 11s 2ms/step - loss: 0.7725

In [10]:


test_images = features_preprocessing_pileline.fit_transform(test_images)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int64)
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)


2024-01-28 18:18:00.388577: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 347.89MiB (rounded to 364788992)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-01-28 18:18:00.388601: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-01-28 18:18:00.388608: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 31, Chunks in use: 29. 7.8KiB allocated for chunks. 7.2KiB in use in bin. 880B client-requested in use in bin.
2024-01-28 18:18:00.388612: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2024-01-28 18:18:00.388615: I external

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.